In [41]:
# 导入必要的库
import os
import torch
from transformers import AutoTokenizer, AutoModel, GPT2LMHeadModel, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline

In [42]:

# 设置设备：优先使用GPU，否则使用CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用设备: {device}")
if device == "cuda":
    print(f"GPU名称: {torch.cuda.get_device_name(0)}")   

使用设备: cpu


### 1. 配置向量嵌入模型 (all-MiniLM-L6-v2)

In [43]:
def embedding_model():
    """配置文本嵌入模型，用于将文本转换为向量"""
    # 定义模型名称
    model_name = "./models/all-MiniLM-L6-v2"
    
    # 配置模型参数
    model_kwargs = {'device': device}  # 指定运行设备
    
    # 配置编码参数：归一化嵌入向量，便于计算余弦相似度
    encode_kwargs = {'normalize_embeddings': True}
    
    # 初始化HuggingFaceEmbeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    return embeddings

### 2. 配置向量数据库 (Chroma)

In [44]:
def vector_database(embeddings, documents=None, persist_directory="./chroma_db"):
    """
    配置Chroma向量数据库
    
    参数:
        embeddings: 嵌入模型
        documents: 如果提供了文档，存储文档，创建新的向量数据库,，如果为None则加载已有数据库
        persist_directory: 数据库持久化存储路径
    """
    # 如果提供了文档，则创建新的向量数据库
    if documents:
        db = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            persist_directory=persist_directory
        )
        # 持久化数据库
        db.persist()
        print(f"已创建并持久化向量数据库，存储路径: {persist_directory}")
    else:
        # 否则加载已有的向量数据库
        db = Chroma(
            persist_directory=persist_directory,
            embedding_function=embeddings
        )
        print(f"已加载现有向量数据库，存储路径: {persist_directory}")
    
    return db


### 3. 配置生成模型 (GPT2)

In [ ]:
def generation_model(max_new_tokens=200, temperature=0.7):
    """
    配置GPT2生成模型
    
    参数:
        max_new_tokens: 生成文本的最大长度
        temperature: 控制生成文本的随机性，值越高越随机
    """
    # 加载GPT2模型和分词器
    model_name = "./models/gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
    # GPT2默认没有pad_token，设置为eos_token
    tokenizer.pad_token = tokenizer.eos_token
    
    # 创建文本生成管道
    generation_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=0 if device == "cuda" else -1,  # 0表示使用第一个GPU，-1表示使用CPU
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,  # 启用采样，使生成结果更多样化
        repetition_penalty=1.5 # 增强重复惩罚，防止 “重复生成同一句话”；
    )
    
    # 将管道包装为LangChain的LLM
    llm = HuggingFacePipeline(pipeline=generation_pipeline)
    
    return llm


### 4. 文档加载与预处理

In [46]:
def load_and_process_documents(file_path, chunk_size=500, chunk_overlap=50):
    """
    加载文档并进行预处理（分割为小块）
    参数:
        file_path: 文档文件路径
        chunk_size: 每个文本块的大小
        chunk_overlap: 文本块之间的重叠大小
    """
    # 检查文件是否存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文档文件不存在: {file_path}")
    
    # 加载文档
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    print(f"已加载文档，原始长度: {len(documents[0].page_content)} 字符")
    
    # 初始化文本分割器
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]  # 分割符优先级
    )
    
    # 分割文档
    split_documents = text_splitter.split_documents(documents)
    print(f"文档分割完成，共生成 {len(split_documents)} 个文本块")
    
    return split_documents


### 5. 查询嵌入、向量检索、提示词、生成回答

In [ ]:
def build_rag_chain(db, llm):
    """
    构建检索增强生成(RAG)管道
    
    参数:
        db: 向量数据库
        llm: 语言生成模型
    """
    # 创建检索器，从数据库中检索相关文档
    # k=3 表示检索最相关的3个文档片段
    retriever = db.as_retriever(search_kwargs={"k": 3})
    
    # 定义提示模板，将检索到的上下文和问题结合起来
    prompt_template = """请严格根据以下上下文回答问题，不要添加上下文外的信息。
    如果上下文没有答案，只说“根据提供的信息，我无法回答这个问题”。

    上下文:{context}
    问题:{question}
    回答:"""
    
    # 创建提示模板
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    
    # 构建RAG链
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # 将所有检索到的文档"填充"到提示中
        retriever=retriever,
        return_source_documents=True,  # 返回用于生成答案的源文档
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    return rag_chain


### 6. 执行

In [ ]:
def main():
    # 配置文件路径和参数
    document_path = "sample_documents.txt"  # 样本文档路径
    persist_directory = "./chroma_db"      # 向量数据库存储路径
    
    # 检查是否存在样本文档，如果不存在则创建一个简单的样本
    if not os.path.exists(document_path):
        with open(document_path, "w", encoding="utf-8") as f:
            f.write("人工智能（AI）是计算机科学的一个分支，致力于创造能够模拟人类智能的系统。\n\n")
            f.write("机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并改进，而无需显式编程。\n\n")
            f.write("深度学习是机器学习的一个子集，它使用多层神经网络来处理复杂的数据。\n\n")
            f.write("自然语言处理（NLP）是人工智能的一个领域，专注于使计算机能够理解、解释和生成人类语言。\n\n")
            f.write("GPT（生成式预训练转换器）是一种基于Transformer架构的语言模型，由OpenAI开发。\n\n")
            f.write("计算机视觉是人工智能的一个领域，致力于使计算机能够理解图像和视频内容。\n\n")
            f.write("强化学习是一种机器学习方法，通过与环境交互并获得奖励来学习最佳行动策略。\n\n")
            f.write("AI的应用领域包括医疗、金融、教育、交通等，正在改变我们的生活和工作方式。")
            f.write ("目标检测是计算机视觉的任务，用于识别图像中多个目标的位置和类别。\n\n")
            f.write ("图像分割将图像划分为不同的区域，每个区域对应一个语义类别。\n\n")
            f.write ("强化学习中的智能体通过试错学习最优策略，最大化累积奖励。\n\n")
            f.write ("Q-learning 是一种无模型强化学习算法，用于学习动作价值函数。\n\n")
            f.write ("深度强化学习结合了深度学习和强化学习，能处理高维状态空间。\n\n")
            f.write ("多智能体系统研究多个智能体如何协同工作以实现共同目标。\n\n")
            f.write ("专家系统是早期 AI 的一种形式，模拟人类专家的决策过程。\n\n")
            f.write ("模糊逻辑允许计算机处理不精确的信息，类似于人类的近似推理。\n\n")
            f.write ("进化算法受生物进化启发，通过选择、交叉和变异操作寻找最优解。\n\n")
            f.write ("知识表示是 AI 的基础问题，研究如何形式化表示知识以便计算机处理。\n\n")
            f.write ("推理引擎使用规则和事实进行逻辑推理，得出新的结论。\n\n")
            f.write ("大语言模型（LLM）拥有数十亿参数，能理解和生成类人文本。\n\n")
            f.write ("提示工程是设计有效的输入提示以引导 AI 模型产生期望输出的技术。\n\n")
            f.write ("AI 可解释性研究如何使 AI 模型的决策过程变得透明和可理解。\n\n")
            f.write ("鲁棒性指 AI 模型在面对噪声或异常数据时保持性能稳定的能力。\n\n")
            f.write ("泛化能力是模型将从训练数据学到的知识应用到新数据的能力。\n\n")
            f.write ("过拟合是模型过度适应训练数据，导致在新数据上表现不佳的现象。\n\n")
            f.write ("欠拟合指模型未能充分学习训练数据中的模式，性能较差。\n\n")
            f.write ("正则化技术用于防止模型过拟合，如 L1 正则化和 L2 正则化。\n\n")
            f.write ("批量归一化是一种训练深度神经网络的技术，能加速收敛并提高稳定性。\n\n")
            f.write ("dropout 是一种防止过拟合的技术，在训练时随机丢弃部分神经元。\n\n")
            f.write ("注意力机制使模型能聚焦于输入数据中最相关的部分。\n\n")
            f.write ("AI 安全研究如何防止 AI 系统被滥用或出现意外的有害行为。")
        print(f"已创建样本文档: {document_path}")
    
    try:
        # 步骤1: 加载和处理文档
        documents = load_and_process_documents(document_path)
        
        # 步骤2: 设置向量嵌入模型
        embeddings = embedding_model()
        
        # 步骤3: 设置向量数据库
        # 如果数据库目录不存在，则创建新的；否则加载现有数据库
        if not os.path.exists(persist_directory):
            db = vector_database(embeddings, documents, persist_directory)
        else:
            db = vector_database(embeddings, persist_directory=persist_directory)
        
        # 步骤4: 设置生成模型
        llm = generation_model(max_new_tokens=150, temperature=0.3) # 降低随机性，避免乱码
        
        # 步骤5: 构建RAG链
        rag_chain = build_rag_chain(db, llm)
        
        # 步骤6: 测试RAG系统
        print("\n===== RAG系统测试 =====")
        test_questions = [
            "什么是人工智能？",
            "机器学习和深度学习有什么关系？",
            "GPT是什么？",
            "AI在哪些领域有应用？",
            "什么是量子计算？",  # 这个问题在样本文档中没有答案
        
        ]
        
        for question in test_questions:
            print(f"\n问题: {question}")
            result = rag_chain({"query": question})
            
            print("回答:", result["result"])
            
            # 打印使用的源文档
            print("\n使用的参考文档片段:")
            for i, doc in enumerate(result["source_documents"], 1):
                print(f"片段 {i}: {doc.page_content[:100]}...")
        
    except Exception as e:
        print(f"执行过程中出现错误: {str(e)}")


In [49]:

# 当脚本直接运行时执行主函数
if __name__ == "__main__":
    main()


已创建样本文档: sample_documents.txt
已加载文档，原始长度: 1060 字符
文档分割完成，共生成 3 个文本块


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


已加载现有向量数据库，存储路径: ./chroma_db


Device set to use cpu
Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



===== RAG系统测试 =====

问题: 什么是人工智能？


...

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


回答: 使用以下提供的上下文来回答问题。如果在上下文中找不到答案，
    请说明"根据提供的信息，我无法回答这个问题"。

    上下文:
    人工智能（AI）是计算机科学的一个分支，致力于创造能够模拟人类智能的系统。

机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并改进，而无需显式编程。

深度学习是机器学习的一个子集，它使用多层神经网络来处理复杂的数据。

自然语言处理（NLP）是人工智能的一个领域，专注于使计算机能够理解、解释和生成人类语言。

GPT（生成式预训练转换器）是一种基于Transformer架构的语言模型，由OpenAI开发。

计算机视觉是人工智能的一个领域，致力于使计算机能够理解图像和视频内容。

强化学习是一种机器学习方法，通过与环境交互并获得奖励来学习最佳行动策略。

AI的应用领域包括医疗、金融、教育、交通等，正在改变我们的生活和工作方式。

    问题:
    GPT是什么？

    回答:

·机器学习、可以和篾到找拾痨。

·机器学习、可以和篾到找拾痨。

·机器学习、可以和篾到找拾痨。

·机器学习、可以和篾到找拾痨。

·机器学习、�

使用的参考文档片段:
片段 1: 人工智能（AI）是计算机科学的一个分支，致力于创造能够模拟人类智能的系统。

机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并改进，而无需显式编程。

深度学习是机器学习的一个子集，它...

问题: AI在哪些领域有应用？


Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


回答: 使用以下提供的上下文来回答问题。如果在上下文中找不到答案，
    请说明"根据提供的信息，我无法回答这个问题"。

    上下文:
    人工智能（AI）是计算机科学的一个分支，致力于创造能够模拟人类智能的系统。

机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并改进，而无需显式编程。

深度学习是机器学习的一个子集，它使用多层神经网络来处理复杂的数据。

自然语言处理（NLP）是人工智能的一个领域，专注于使计算机能够理解、解释和生成人类语言。

GPT（生成式预训练转换器）是一种基于Transformer架构的语言模型，由OpenAI开发。

计算机视觉是人工智能的一个领域，致力于使计算机能够理解图像和视频内容。

强化学习是一种机器学习方法，通过与环境交互并获得奖励来学习最佳行动策略。

AI的应用领域包括医疗、金融、教育、交通等，正在改变我们的生活和工作方式。

    问题:
    AI在哪些领域有应用？

    回答:

但用领域有应用？

但用领域有应用？

但用领域有应用？

但用领域有应用？

但用领域有应用？

但用领域有应用？

但用领域有应用？

�

使用的参考文档片段:
片段 1: 人工智能（AI）是计算机科学的一个分支，致力于创造能够模拟人类智能的系统。

机器学习是人工智能的一个重要子领域，它使计算机能够从数据中学习并改进，而无需显式编程。

深度学习是机器学习的一个子集，它...

问题: 什么是量子计算？


KeyboardInterrupt: 